# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124656e+02     1.716835e+00
 * time: 0.09519290924072266
     1     1.083924e+01     1.025729e+00
 * time: 0.9645860195159912
     2    -1.187896e+01     1.229568e+00
 * time: 1.109678030014038
     3    -3.410361e+01     9.136520e-01
 * time: 1.2672820091247559
     4    -4.751714e+01     6.461960e-01
 * time: 1.418267011642456
     5    -5.681157e+01     2.321287e-01
 * time: 1.5765550136566162
     6    -5.968207e+01     1.990874e-01
 * time: 1.6955249309539795
     7    -6.086193e+01     7.794479e-02
 * time: 1.805898904800415
     8    -6.137427e+01     4.515507e-02
 * time: 1.9260268211364746
     9    -6.167262e+01     3.743134e-02
 * time: 2.0594968795776367
    10    -6.184428e+01     2.872932e-02
 * time: 2.1861038208007812
    11    -6.199456e+01     2.247265e-02
 * time: 2.302367925643921
    12    -6.208013e+01     1.545385e-02
 * time: 2.423360824584961
    13    -6.212866e+01     1.447207e-02
 * time: 2.54450297355651

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557672
    AtomicNonlocal      14.8522637
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666460597
